In [1]:
from fastai.tabular import * 
import pandas as pd
import sklearn
from matplotlib import pyplot as plt


In [2]:
crime_property_light_population = pd.read_csv("./model_data/crime_light_density.csv").iloc[:,1:]
crime_property_light_population.head()

,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,zipcode,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density
0,Auto Theft,E18,N,2019-10-13 09:28:24,2019,10,Sunday,9,LINCOLN ST,42.259518,-71.121563,2136,6206.9,455.644447,21.073507,0.437849
1,Auto Theft,E18,N,2019-10-12 20:11:26,2019,10,Saturday,20,METROPOLITAN AVE,42.262092,-71.116710,2136,6206.9,455.644447,21.073507,0.437849
2,Auto Theft,D14,N,2019-10-12 15:12:43,2019,10,Saturday,15,ALLSTON ST,42.352375,-71.135096,2134,16212.0,2250.588503,63.554783,0.582893
3,Auto Theft,D4,N,2019-10-12 04:41:52,2019,10,Saturday,4,SAINT JAMES AVE,42.349476,-71.076402,2116,32724.1,14799.401260,480.463801,0.656585
4,Auto Theft,A15,N,2019-10-02 08:08:49,2019,10,Wednesday,8,N MEAD ST,42.381846,-71.066551,2129,12192.0,278.377510,165.249738,0.678941


In [4]:
predictors = ['OFFENSE_CODE_GROUP','SHOOTING', 'MONTH','HOUR', 'DAY_OF_WEEK', \
              'Population density (per square mile of land area)', \
              'BLDG_VAL', 'LAND_VAL',\
              'light_density','zipcode']
df = crime_property_light_population[predictors]
df = df.dropna()
df.head()

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,HOUR,DAY_OF_WEEK,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,zipcode
0,Auto Theft,N,10,9,Sunday,6206.9,455.644447,21.073507,0.437849,2136
1,Auto Theft,N,10,20,Saturday,6206.9,455.644447,21.073507,0.437849,2136
2,Auto Theft,N,10,15,Saturday,16212.0,2250.588503,63.554783,0.582893,2134
3,Auto Theft,N,10,4,Saturday,32724.1,14799.401260,480.463801,0.656585,2116
4,Auto Theft,N,10,8,Wednesday,12192.0,278.377510,165.249738,0.678941,2129


In [6]:
#top5 = df['OFFENSE_CODE_GROUP'].value_counts().index[:5]
top5= ['Auto Theft','Motor Vehicle Accident Response', 'Larceny','Robbery','Residential Burglary']
#cut the df
df = df[df['OFFENSE_CODE_GROUP'].isin(top5)]
#enc = OneHotEncoder(handle_unknown='ignore')

In [7]:
top5


['Auto Theft',
 'Motor Vehicle Accident Response',
 'Larceny',
 'Robbery',
 'Residential Burglary']

In [8]:
#df_cat = enc.fit_transform(df[["MONTH","SHOOTING","DAY_OF_WEEK"]]).toarray()

#df_cat = pd.DataFrame(df_cat, columns=enc.get_feature_names(['MONTH','SHOOTING', 'DAY_OF_WEEK']))
#df = pd.concat([df, df_cat], axis=1)
#df = df.dropna()

df.head()

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,HOUR,DAY_OF_WEEK,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,zipcode
0,Auto Theft,N,10,9,Sunday,6206.9,455.644447,21.073507,0.437849,2136
1,Auto Theft,N,10,20,Saturday,6206.9,455.644447,21.073507,0.437849,2136
2,Auto Theft,N,10,15,Saturday,16212.0,2250.588503,63.554783,0.582893,2134
3,Auto Theft,N,10,4,Saturday,32724.1,14799.401260,480.463801,0.656585,2116
4,Auto Theft,N,10,8,Wednesday,12192.0,278.377510,165.249738,0.678941,2129


In [9]:
len(df)

90014

In [10]:
procs = [FillMissing, Categorify, Normalize]

In [11]:
valid_idx = range(len(df)-10000, len(df))

In [12]:
dep_var = 'OFFENSE_CODE_GROUP'
cat_names = ["MONTH","SHOOTING","DAY_OF_WEEK","HOUR"]

data = TabularDataBunch.from_df('./model_data', df, dep_var, \
                                valid_idx=valid_idx, procs=procs, \
                                cat_names=cat_names)

In [13]:
print(data.train_ds.cont_names) 

['Population density (per square mile of land area)', 'LAND_VAL', 'zipcode', 'BLDG_VAL', 'light_density']


In [14]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.166448,1.294135,0.485500,00:08
1,1.170959,1.285450,0.487200,00:09
2,1.153296,1.280289,0.487300,00:09
3,1.169160,1.271143,0.489300,00:10
4,1.158641,1.273442,0.489400,00:10
